# Table of Contents
* [Submitting HITs](#Submitting-HITs)
	* [building html](#building-html)
		* [code](#code)
	* [Building hits from files](#Building-hits-from-files)
	* [load dataset](#load-dataset)
* [submit](#submit)
* [review](#review)
* [Messaging workers](#Messaging-workers)
* [Accepting and deleting HITs... careful with these](#Accepting-and-deleting-HITs...-careful-with-these)
* [pay bonuses](#pay-bonuses)


In [299]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2


import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc dark solarized
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['figure.figsize'] = (16.0, 10.0)


import re
import pickle
import boto
from copy import deepcopy
import json
import os
import jinja2
import random
from tqdm import tqdm

from IPython.core.display import HTML

from boto.mturk.qualification import PercentAssignmentsApprovedRequirement, Qualifications, Requirement

from keysTkingdom import mturk_ai2
from keysTkingdom import aws_tokes
from keysTkingdom import mturk_aristo


from amt_utils.mturk import MTurk
from amt_utils.mturk import pickle_this, unpickle_this

# Setup

## estimate cost

In [312]:
cost_per_hit = 0.04
n_turkers_per_hit = 3
n_hits_per_video = 3

n_videos_low = 200
n_videos_high = 50000

low = cost_per_hit * n_turkers_per_hit * n_videos_low * n_hits_per_video
high = cost_per_hit * n_turkers_per_hit * n_videos_high * n_hits_per_video
print(str(low) + ' - $' + str(high))

72.0 - $18000.0


## load dataset

In [117]:
s3_base_path = 'https://s3-us-west-2.amazonaws.com/ai2-vision-animation-gan/annotation_data/still_frames/'

data_file_path = '../../build_dataset/ds_interface_2_skeleton.json'

with open(data_file_path, 'r') as f:
    ds_skeleton = json.load(f)

stills_to_annotate = []
for clip in ds_skeleton:
    for still in clip['keyFrames']:
        stills_to_annotate.append(still)

In [306]:
to_skip = unpickle_this('./skip_redo.pkl')

In [307]:
skip_stills_to_annotate = [still for still in stills_to_annotate if still not in to_skip]

In [308]:
len(skip_stills_to_annotate)

593

# Submitting HITs

In [309]:
import jinja2
import argparse

from jinja2 import Environment, FileSystemLoader

from amt_utils.flintstones import prepare_hit
from amt_utils.flintstones import generate_task_page

In [328]:
turk_account = mturk_ai2
rw_host='mechanicalturk.amazonaws.com'
amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key, host=rw_host)
# amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key)

amt_con.get_account_balance()

$1,669.95

In [313]:
static_params = {
    'title': "Annotate characters from an animation frame",
    'description': "Draw bounding boxes and label characters appearing in a image",
    'keywords': ['animation', 'image', 'bounding box', 'image annotation'],
    'frame_height': 1000,
    'amount': 0.05,
    'duration': 3600 * 1,
    'lifetime': 3600 * 24 * 2,
    'max_assignments': 3,
}

In [319]:
build_hit_group = [prepare_hit(s3_base_path, still, static_params) for still in skip_stills_to_annotate]

In [329]:
len(build_hit_group)

593

In [330]:
hit_group = [amt_con.create_html_hit(single_hit) for single_hit in tqdm(build_hit_group)]

100%|██████████| 593/593 [02:16<00:00,  4.79it/s]


In [331]:
# amt_con.delete_all_hits()

# Retrieve results

In [236]:
from amt_utils.flintstones import get_assignments
from amt_utils.flintstones import filter_hits_by_date
from amt_utils.flintstones import filter_hits_by_completion
from amt_utils.flintstones import filter_hits_by_status
from amt_utils.flintstones import get_completed_hits
from amt_utils.mturk import pickle_this

In [335]:
all_hits = amt_con.get_all_hits()

In [322]:
hit_series = pd.Series([hit.CreationTime for hit in all_hits])

In [361]:
recent_hits = filter_hits_by_date(all_hits, 4, 20)
recent_hits = filter_hits_by_status(recent_hits)

In [362]:
len(recent_hits)

528

In [363]:
results = get_assignments(amt_con.connection, tqdm(recent_hits))

100%|██████████| 528/528 [01:02<00:00,  8.52it/s]


In [364]:
pickle_this(results, 'turker_batch_3.pkl')

# Interact with workers

In [ ]:
subject = "More science book annotation HITs are available"
message = """
Hello, 

If you're receiving this message you were among the top performers on the first group HITs I submitted.
I've submitted another group of HITs, with more to follow in the next few days. 
This task is slightly different from the first, so please review the new instructions before jumping in.

Happy to get any feedback you might have for the new HITs.
"""

In [15]:
# _ = mturk.notify_workers(good_workers[:20], subject, message)

In [16]:
def pay_bonuses(correct_assignments):
    bonus_ammount = boto.mturk.price.Price(0.30)
    bonus_reason_template = 'For correctly answering the science question given in assignment {}.'
    for worker_ans in correct_assignments:
        bonus_reason = bonus_reason_template.format(str(worker_ans['assignment_id']))
        mturk.grant_bonus(worker_ans['wid'], worker_ans['assignment_id'], bonus_ammount, bonus_reason)

# Accepting and deleting HITs... careful with these

Uncomment only when ready to accept or delete hits

reject assignments carefully

In [332]:
# assignments =[]
# for assignment_triple in list(results.values()):
#     assignments.extend(assignment_triple)

# len(assignments)

# for assignment in tqdm(assignments):
#     try:
#         amt_con.approve_assignment(assignment)
#     except:
#         pass

In [ ]:
# _ = [amt_con.disable_hit(hit) for hit in hits_today]

In [ ]:
# number_rejected_assignments, number_rejected_workers = amt_util.reject_assignments(mturk, workers_to_ban, combined_consensus_with_workerid_df)
# print 'rejecting ' + str(number_rejected_assignments) + ' assignments' + ' from ' + str(number_rejected_workers) + ' workers'

In [ ]:
flattened_assignments = [item for sublist in assignment_results_current_batch.values() for item in sublist]
len(flattened_assignments)
# amt_util.get_assignment_statuses(assignment_results_current_batch)

In [ ]:
amt_util.get_assignment_statuses(assignment_results_current_batch)

In [ ]:
len(flattened_assignments)

In [ ]:
amt_util.accept_hits(mturk, flattened_assignments)